In [ ]:
pip install sentence_transformers

## performing clustering on large scale dataset.
#### In this example, we download a large set of questions from Quora and then find similar questions in this set.

##### Parmas to consider:
##### 1. Threshold : The higher the treshold the higher the similarity
##### 2. 'min_community_size': Only communities with at least a certain number of sentences will be returned.

#### import liberties

In [ ]:
from sentence_transformers import SentenceTransformer, util
import os
import csv
import time

#####Inital the embedding model

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

#### Download the Quora Duplicate Questions Dataset

In [ ]:
url = "http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv"
dataset_path = "quora_duplicate_questions.tsv"
# We limit our corpus to only the first 50k questions
max_corpus_size = 50000

#### Check if the dataset exists. If not, download and extract


In [ ]:
if not os.path.exists(dataset_path):
    print("Download dataset")
    util.http_get(url, dataset_path)

Download dataset


  0%|          | 0.00/58.2M [00:00<?, ?B/s]

#### Get all unique sentences from the file

In [ ]:
corpus_sentences = set()
with open(dataset_path, encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_MINIMAL)
    for row in reader:
        corpus_sentences.add(row['question1'])
        corpus_sentences.add(row['question2'])
        if len(corpus_sentences) >= max_corpus_size:
            break

corpus_sentences = list(corpus_sentences)
print("Encode the corpus. This might take a while")
corpus_embeddings = model.encode(corpus_sentences, batch_size=64, show_progress_bar=True, convert_to_tensor=True)
print("Start clustering")
start_time = time.time()

Encode the corpus. This might take a while


Batches:   0%|          | 0/782 [00:00<?, ?it/s]

Start clustering


###Two parameters to tune:
#####min_cluster_size: Only consider cluster that have at least 25 elements
#####threshold: Consider sentence pairs with a cosine-similarity larger than threshold as similar

In [ ]:
clusters = util.community_detection(corpus_embeddings, min_community_size=25, threshold=0.75)
print("Clustering done after {:.2f} sec".format(time.time() - start_time))

Clustering done after 58.28 sec


####Print for all clusters the top 3 and bottom 3 elements

In [ ]:
for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster[0:3]:
        print("\t", corpus_sentences[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", corpus_sentences[sentence_id])


Cluster 1, #103 Elements 
	 What is the best way to talk English fluently?
	 How do I talk English fluently?
	 What can I do to improve my English Grammar?
	 ...
	 How can I improve my communication skills in English?
	 How can I learn English well？?
	 How can I become fluent in English faster?

Cluster 2, #86 Elements 
	 How can I make money online for job?
	 How can I start earning money online ?
	 What are some different ways to make money online, excluding selling things?
	 ...
	 How can an apprentice programmer make money online?
	 What are the best websites for making money online?
	 What are the various ways through which one can earn money online?

Cluster 3, #82 Elements 
	 What are the impacts on Indian Rupee value after demonetizing Rs. 500 or Rs. 1000 notes in market as economic stand point?
	 How it will effect Indian economy after banning rs.500 and rs.1000 currency bills?
	 How Indian economy got affected after ban of 500 1000 notes?
	 ...
	 How would demonetizing 500 a